In [1]:
import pandas as pd
import numpy as np
import os
from collections import OrderedDict, Counter, defaultdict, namedtuple
from enum import Enum
from copy import deepcopy
from scipy.optimize import minimize
from itertools import combinations, chain
import re
import math

from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from matplotlib.figure import Figure
import matplotlib.path as mpath
import matplotlib.patches as mpatches

In [2]:
# ENUMS def
class rEnum:
    def __init__(self, name, *str_values):
        assert type(name) == str, 'Name need be str type'
        assert bool(re.fullmatch(r'\w+', name)), 'Name have to consists of alphas, nums and _'
        assert not name[0].isdigit(), 'Name cannot begins from digit'
        # add 
        assert all(map(lambda i: type(i) == str, str_values)), 'Keys need be str type'
        assert len(set(str_values)) == len(str_values), 'Enum need have unique keys'
        for str_val in str_values:
            self.__dict__[str_val] = '{}.{}'.format(name,str_val)
        #assert not name in globals().keys(), 'Name {} already in globals'
        globals()[name] = self
    
    @property
    def possibleValues(self):
        return [eval(i) for i in self.__dict__.values()]

rEnum ('E_MergePolicy','Direct', \
                       'Side')

rEnum ('E_FieldAxis','X', \
                     'Y')

rEnum ('E_SigColor','R', \
                    'B', \
                    'W', \
                    'Y', \
                    'G')

rEnum ('E_SigType','Train', \
                   'Man', \
                   'AB', \
                   'NeighbourStationInTrain', \
                   'Repeat', \
                   'Warning')

rEnum ('E_Direct','Plus', \
                  'Minus')

rEnum ('E_IsolSegmType','STREL', \
                        'BESSTREL', \
                        'STREL_ON_STRVP', \
                        'BESSTREL_ON_STRVP', \
                        'PUT', \
                        'INDIC', \
                        'ABTC', \
                        'PABRC', \
                        'FICTIVE')

rEnum ('E_RefreshData','ALL', \
                       'NEW_CS', \
                       'NEW_POINT', \
                       'NEW_LINE', \
                       'NEW_SEGMENT')

In [ ]:
class CellEval:
    def __init__(self):
        self.candidateValue = None
        self.value = None
        self.evalStatus = False
        self.expectedType = None #

class ComplexAttrib:
    def __init__(self):
        self.name = None 
        self.isCompulsory = False 
        self.cell = CellEval()
        self.expectedType = None #
        self.isIterable = False
        self.isRepeatable = False

class Node:
    def __init__(self):
        self.parents = []
        self.childs = []
    
class Tree:
    def __init__(self):
        self.nodes = None

In [ ]:
class AttribsGroup:
    def __init__(self):
        self.mainAttr = None
        self.attribs = None
        self.isActive = None
    
    def activate(self):
        pass
    
    def deactivate(self):
        pass
    
class CreationMethodsGroup:
    def __init__(self):
        self.mainMethod = None
        self.activeMethod = None
        self.methods = {}
        
    def addMethod(self, methodName, attrGroup):
        self.methods[methodName] = attrGroup
        

In [ ]:
class A:
    creationMethods = ['cm1', 'cm2', 'cm3']
    
    

In [3]:
class CoordToBasisTransformation:
    def __init__(self, shift, direction):
        self.Shift = shift
        self.Direction = direction
        
    def __repr__(self):
        return '{}({},{})'.format(self.__class__.__name__, self.Shift, self.Direction)
        
    @property
    def Shift(self):
        return self._Shift
    @Shift.setter
    def Shift(self, value):
        assert isinstance(value, (int, float)), 'Expected type is int or float'
        self._Shift = value
        
    @property
    def Direction(self):
        return self._Direction
    @Direction.setter
    def Direction(self, value):
        assert abs(int(value)) == 1, 'Expected value is 1 or -1'
        self._Direction = value

In [4]:
class FieldCoord:
    def __init__(self, value, cs, axis = E_FieldAxis.X):
        self.Cs = cs
        self.Axis = axis
        self._X = None
        self._Y = None
        self.FloatCoord = value
        
    def __repr__(self):
        return '{}(({},{}),{})'.format(self.__class__.__name__, self._X, self._Y, self.Cs) 
    
    @property
    def X(self):
        return self._X
    @X.setter
    def X(self, value):
        assert type(value) in [str,int,float], 'Expected type is str, int or float, given {}'.format(value)
        if type(value) in [int,float]:
            self._X = CE.simpleSingleCast(value, float)
        else:
            assert bool(re.fullmatch(r'PK_\d+\+\d+', value)), 'Coord format is PK_xxxx+yyy, given {}'.format(value)
            plus_pos = value.find('+')
            self._X = float(int(value[3:plus_pos]) * 100 + int(value[plus_pos + 1:]))
    
    @property
    def Y(self):
        return self._Y
    @Y.setter
    def Y(self, value):
        self._Y = CE.simpleSingleCast(value, float)
        
    @property
    def FloatCoord(self):
        return self._FloatCoord
    @FloatCoord.setter
    def FloatCoord(self, value):
        assert type(value) in [str,int,float,tuple], 'Expected type is tuple, str, int or float, given {}'.format(value)
        if not type(value) == tuple:
            if self.Axis == E_FieldAxis.X:
                self.X = value
            else:
                self.Y = value
        else:
            assert len(value) == 2, 'Expected 2 coords: {}'.format(value)
            self.X = value[0]
            self.Y = value[1]
        self._FloatCoord = (self.X, self.Y)
    @property
    def PicketCoord(self):
        assert self.X.is_integer(), 'Cannot convert float {} to picket value'.format(self.X)
        int_val = CE.simpleSingleCast(self.X, int)
        return 'PK_{}+{}'.format(int_val//100, int_val%100)
        
    @property
    def Cs(self):
        return self._Cs
    @Cs.setter
    def Cs(self, value):
        self._Cs = CE.simpleSingleCast(value, CoordinateSystem)
    
    @property
    def Axis(self):
        return self._Axis
    @Axis.setter
    def Axis(self, value):
        self._Axis = CE.enumSimpleCast(value, E_FieldAxis)
        
    def __add__(self, other):
        assert type(other) in [int, float], 'Expected num. type of 2nd arg'
        return FieldCoord((self.X + other, self.Y), self.Cs)
    
    def __radd__(self, other):
        return add(self, other)
    
    #def __iadd__(self, other):
    #    return add(self, other)
    
    def __sub__(self, other):
        #print(other)
        assert isinstance(other, FieldCoord) or (type(other) in [int, float]), 'Expected type of 2nd arg - FieldCoord or num.'
        if type(other) in [int, float]:
            return FieldCoord((self.X - other, self.Y), self.Cs)
        else:
            assert self.Cs == other.Cs , 'Coords for sub need to be in same CS'
            return self.X - other.X 
        
    def toBasis(self):
        basisCs = self.Cs.Basis.MainCoordSystem
        new_X = self.Cs.CoordToBasisTransformation.Shift + self.Cs.CoordToBasisTransformation.Direction * self.X
        return FieldCoord((new_X, self.Y), basisCs)
        #pass
        #assert isinstance(other, CoordinateSystem), 'Expected type of 2nd arg - CoordinateSystem'
        

#a = FieldCoord('PK_129+23',CoordinateSystem_Basis_1)

In [5]:
#FieldCoord

In [5]:
# SUPPORTING NON-STANDARD CLASSES

# Defaults
class Defaults:
    
    def __init__(self):
        self.InterRailDist = 5.3
        
DFLT = Defaults()  

In [6]:
class CastEvaluations:
    
    def inputIsStr(self, in_value):
        return type(in_value) == str
    
    def intToFloat(self, in_value, need_type):
        if (need_type == float) and (type(in_value) == int):
            return float(in_value)
        else:
            return in_value
    
    def floatToInt(self, in_value, need_type):
        if (need_type == int) and (type(in_value) == float):
            assert in_value.is_integer(), 'Expected integer value, given is {}'.format(in_value)
            return int(in_value)
        else:
            return in_value
        
    def floatIntEquivalence(self, in_value, need_type):
        in_value = self.intToFloat(in_value, need_type)
        in_value = self.floatToInt(in_value, need_type)
        return in_value
    
    def emptyStrAsNone(self, in_value):
        if (type(in_value) == str) and (in_value.isspace() or not in_value):
            return None
        else:
            return in_value
    
    def autoCast(self, in_value):
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            if self.inputIsStr(in_value):
                return eval(in_value)
            else:
                return in_value
        else:
            return in_value
        
    def strCast(self, in_value):
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            assert self.inputIsStr(in_value), 'Input type is not a str : {}'.format(in_value)
        return in_value
            
    def simpleSingleCast(self, in_value, need_type, floatIntEquiv = True): # int, float, bool, simple class
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            if self.inputIsStr(in_value):
                out_value = eval(in_value)
            else:
                out_value = in_value
            if floatIntEquiv:
                out_value = self.floatIntEquivalence(out_value, need_type)
            assert issubclass(type(out_value), need_type), 'Cannot convert to {} : {}'.format(need_type.__name__, out_value)
            return out_value
        else:
            return in_value
        
    def simpleIterableCast(self, in_value, need_iterable_type, need_each_type, need_count=None, floatIntEquiv = True):
        assert need_iterable_type in [set,list,tuple], 'Type iterable {} is not supported'.format(need_iterable_type)
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            if self.inputIsStr(in_value):
                out_value = eval(in_value)
            else:
                out_value = in_value
            assert type(out_value) == need_iterable_type, 'Need type of iterable is {}'.format(need_iterable_type)
            if floatIntEquiv:
                out_value = need_iterable_type([self.floatIntEquivalence(i, need_each_type) for i in out_value])
            assert all(map(lambda i: issubclass(type(i), need_each_type),out_value)), 'Not all elements of class {} : {}'.format(need_each_type.__name__, out_value)
            if not need_count is None:
                assert type(need_count) == int, 'Need_count need to be int, given value is {}'.format(need_count)
                assert len(out_value) == need_count, 'Needed count is {}, given value is {}'.format(need_count,out_value)
            return out_value
        else:
            return in_value
        
    def customCast(self, in_value, func_cast_success):
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            if self.inputIsStr(in_value):
                out_value = eval(in_value)
            else:
                out_value = in_value
            assert func_cast_success(out_value),  'Convert result not satisfy func_cast_success : {}'.format(out_value)
            return out_value
        else:
            return in_value
        
    def enumSimpleCast(self, in_value, enum_of_value):
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            if self.inputIsStr(in_value):
                out_value = eval(in_value)
            else:
                out_value = in_value
            assert out_value in enum_of_value.possibleValues, 'Value {} not in possb. enum vals. {}'.format(out_value,enum_of_value.possibleValues)
            return out_value
        else:
            return in_value
        
    def enumIterableCast(self, in_value, need_iterable_type, enum_of_each_value, need_count=None):
        assert need_iterable_type in [set,list,tuple], 'Type iterable {} is not supported'.format(need_iterable_type)
        in_value = self.emptyStrAsNone(in_value)
        if not in_value is None:
            if self.inputIsStr(in_value):
                out_value = eval(in_value)
            else:
                out_value = in_value
            assert type(out_value) == need_iterable_type, 'Need type of iterable is {}'.format(need_iterable_type)
            assert all(map(lambda i: i in enum_of_each_value.possibleValues, out_value)), 'Not all elements is enum vals {} : {}'.format(need_each_type.__name__, out_value)
            if not need_count is None:
                assert type(need_count) == int, 'Need_count need to be int, given value is {}'.format(need_count)
                assert len(out_value) == need_count, 'Needed count is {}, given value is {}'.format(need_count,out_value)
            return out_value
        else:
            return in_value
        
CE = CastEvaluations()

# Descriptors

In [8]:
# Descriptor classes
'''
class Typed_SimpleSingle:
    def __init__(self, need_type, in_value = None, floatIntEquiv = True):
        self.Need_type = need_type
        self.Fl_Int_Eq = floatIntEquiv
        self.Value = in_value
        
    def __repr__(self):
        return repr(self.Value)
        
    @property
    def Value(self):
        return self._Value
    @Value.setter
    def Value(self, value):
        self._Value = value

class TypedIterable:
    def __init__(self, need_each_type, values=[], need_count=None):
        self.Need_each_type = need_each_type
        assert type(values) in [tuple, list, set], 'Expected init type - iterable'
        self.Need_iterable_type = type(values)
        self.Need_count = need_count
        self.Values = values
        
    @property
    def Values(self):
        return self._Values
    @Values.setter
    def Values(self, values):
        #print(self.Need_iterable_type, self.Need_each_type, self.Need_count, values)
        self._Values = CE.simpleIterableCast(values, self.Need_iterable_type, self.Need_each_type, self.Need_count)
        #print(self._Values)
    
    def append(self, value):
        self._Values.append(CE.simpleSingleCast(value, self.Need_each_type))
        
    def add(self, value):
        self._Values.add(CE.simpleSingleCast(value, self.Need_each_type))
        
    def remove(self, value):
        self._Values.remove(value)
'''

"\nclass Typed_SimpleSingle:\n    def __init__(self, need_type, in_value = None, floatIntEquiv = True):\n        self.Need_type = need_type\n        self.Fl_Int_Eq = floatIntEquiv\n        self.Value = in_value\n        \n    def __repr__(self):\n        return repr(self.Value)\n        \n    @property\n    def Value(self):\n        return self._Value\n    @Value.setter\n    def Value(self, value):\n        self._Value = value\n\nclass TypedIterable:\n    def __init__(self, need_each_type, values=[], need_count=None):\n        self.Need_each_type = need_each_type\n        assert type(values) in [tuple, list, set], 'Expected init type - iterable'\n        self.Need_iterable_type = type(values)\n        self.Need_count = need_count\n        self.Values = values\n        \n    @property\n    def Values(self):\n        return self._Values\n    @Values.setter\n    def Values(self, values):\n        #print(self.Need_iterable_type, self.Need_each_type, self.Need_count, values)\n        self._

In [9]:
#a = TypedIterable(int, ['1','2','3'])

In [10]:
#a = Typed_SimpleSingle(int, 10)

In [11]:
'''
class Meter: #(object)

    def __init__(self, value=0.0):
        self.value = float(value)
    def __get__(self, instance, owner):
        return self.value
    def __set__(self, instance, value):
        self.value = float(value)

class Foot:

    def __get__(self, instance, owner):
        return instance.meter * 3.2808
    def __set__(self, instance, value):
        instance.meter = float(value) / 3.2808

class Distance:
    meter = Meter()
    foot = Foot()
    
a = Distance()
a.foot = 1
a.meter
'''

'\nclass Meter: #(object)\n\n    def __init__(self, value=0.0):\n        self.value = float(value)\n    def __get__(self, instance, owner):\n        return self.value\n    def __set__(self, instance, value):\n        self.value = float(value)\n\nclass Foot:\n\n    def __get__(self, instance, owner):\n        return instance.meter * 3.2808\n    def __set__(self, instance, value):\n        instance.meter = float(value) / 3.2808\n\nclass Distance:\n    meter = Meter()\n    foot = Foot()\n    \na = Distance()\na.foot = 1\na.meter\n'

In [12]:
'''
class A:
    def __init__(self):
        self._a = None
    
    @property
    def a(self):
        return self._a
    @a.setter
    def a(self, value):
        self._a = value

class D:
    def __init__(self):
        self.a = NonNegative(Point)

d = D()
'''

'\nclass A:\n    def __init__(self):\n        self._a = None\n    \n    @property\n    def a(self):\n        return self._a\n    @a.setter\n    def a(self, value):\n        self._a = value\n\nclass D:\n    def __init__(self):\n        self.a = NonNegative(Point)\n\nd = D()\n'

In [13]:
'''
class NonNegative:
    def __init__(self, str_my_type):
        #print('in init', my_type)
        self.str_my_type = str_my_type  # (4)
    def __get__(self, instance, owner): 
        #print('in get', instance, owner) 
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        #print('in set', instance, value)
        self.my_type = eval(self.str_my_type)
        assert type(value) == self.my_type, 'Not needed type'
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value
    def __set_name__(self, owner, name):
        #print('in set name', owner, name)
        self.name = name

#class D:
#    def __init__(self):
#        self.a = NonNegative(Point)
        
class Order:
    price = NonNegative('int')
    quantity = NonNegative('CoordinateSystem') #CoordinateSystem
    
    def __init__(self, name, price_1, quantity_1):
        self._name = name
        self.price = price_1
        self.quantity = quantity_1

    def total(self):
        return self.price * self.quantity
    
#Order.__dict__['price'] = NonNegative(int)
#Order.__dict__['quantity'] = NonNegative(int)

apple_order = Order('apple', 1, 10)
apple_order.total()
# 10
'''

"\nclass NonNegative:\n    def __init__(self, str_my_type):\n        #print('in init', my_type)\n        self.str_my_type = str_my_type  # (4)\n    def __get__(self, instance, owner): \n        #print('in get', instance, owner) \n        return instance.__dict__[self.name]\n    def __set__(self, instance, value):\n        #print('in set', instance, value)\n        self.my_type = eval(self.str_my_type)\n        assert type(value) == self.my_type, 'Not needed type'\n        if value < 0:\n            raise ValueError('Cannot be negative.')\n        instance.__dict__[self.name] = value\n    def __set_name__(self, owner, name):\n        #print('in set name', owner, name)\n        self.name = name\n\n#class D:\n#    def __init__(self):\n#        self.a = NonNegative(Point)\n        \nclass Order:\n    price = NonNegative('int')\n    quantity = NonNegative('CoordinateSystem') #CoordinateSystem\n    \n    def __init__(self, name, price_1, quantity_1):\n        self._name = name\n        self.

In [14]:
#apple_order.price = -10
#apple_order.quantity = -10

# End Descriptors

In [7]:
class NameObj:
    
    def __init__(self, name, obj):
        self.name = name
        self.obj = obj
        
    def __eq__(self, other):
        return (self.name == other.name) and (self.obj == other.obj)
    
    def __hash__(self):
        return hash((self.name,self.obj))

class InstCounter:
    
    def __init__(self):
        self._count = 0
        self._insts = [] #set()
    
    def increaseCount(self):
        self._count += 1
    
    def resetCount(self):
        self._count = 0
        
    def addInst(self, inst):
        self._insts.append(NameObj(inst.Name, inst)) # add
        self.increaseCount()
        
    def removeInst(self, inst, name_which_added = None):
        if name_which_added is None:
            name_which_added = inst.Name
        self._insts.remove(NameObj(name_which_added, inst))
        if not (self.instSize):
            self.resetCount()
    
    @property
    def count(self):
        return self._count
    
    @property
    def insts(self):
        return self._insts
    
    @property
    def instSize(self):
        return len(self._insts)

if 'OI' in globals():
    OI.removeAllClassInstances()

class ObjsInspector:
    
    def __init__(self):
        self._insp_dict = {}
        
    def clsIsRegistered(self, cls):
        return (cls in self._insp_dict.keys())
    
    def registerCls(self, cls):
        if not (self.clsIsRegistered(cls)):
            self._insp_dict[cls] = InstCounter()
    
    def registerObj(self, obj):
        cls_ierarh = reversed(obj.__class__.__mro__[:-1])
        for cls in cls_ierarh:
            if not self.clsIsRegistered(cls):
                self.registerCls(cls)
        #cell_obj_registered = self.getRegCell(obj)
        #if not (cell_obj_registered is None):
        self.removeObj(obj) #cell_obj_registered.obj, cell_obj_registered.name
        self.addObj(obj)
        
    def getNumeration(self, cls):
        self.registerCls(cls)
        return(self._insp_dict[cls].count + 1)
        
    def getRegCell(self, obj):
        cls = obj.__class__
        res = list(filter(lambda x: x.obj == obj, self._insp_dict[cls].insts))
        if res:
            return res[0]
    
    def getInstances(self, cls):
        if not(self.clsIsRegistered(cls)):
            self.registerCls(cls)
        return [x.obj for x in self._insp_dict[cls].insts] # {}
    
    def addObj(self, obj):
        #print('adding obj', obj, obj.Name)
        cls_ierarh = obj.__class__.__mro__[:-1]
        for cls in cls_ierarh:
            self._insp_dict[cls].addInst(obj)
        globals()[obj.Name] = obj
    
    def removeObj(self, obj): #, name_which_added = None
        cell_obj_registered = self.getRegCell(obj)
        if not (cell_obj_registered is None):
            cls_ierarh = reversed(obj.__class__.__mro__[:-1])
            for cls in cls_ierarh:
                self._insp_dict[cls].removeInst(obj, cell_obj_registered.name)
            globals().pop(cell_obj_registered.name)
    
    def removeInstances(self, cls):
        for obj in self.getInstances(cls):
            self.removeObj(obj) #, obj.Name
    
    def removeAllClassInstances(self):
        for cls in self._insp_dict.keys():
            for obj in self.getInstances(cls):
                self.removeObj(obj) #, obj.Name
                
    def checkNameRepeating(self, cls, name_pretend):
        return any(map(lambda x: x.Name == name_pretend, self.getInstances(cls)))

OI = ObjsInspector()

In [8]:
# Base Functionality Provider
class BFP():
    
    def __init__(self,**kwargs):
        if 'Name' in kwargs.keys():
            name_pretend = kwargs['Name']
            prefix = self.__class__.__name__ + '_'
            assert not name_pretend[len(prefix):].isdigit(), 'Not auto-name cannot be (prefix + int); choose other name'
            self.Name = kwargs['Name']
        else:
            self.Name = '{}_{}'.format(self.__class__.__name__, OI.getNumeration(self.__class__))
        OI.registerObj(self)
        
    def __repr__(self):
        return self._Name
        
    @property
    def Name(self):
        return self._Name
    
    @Name.setter
    def Name(self, name_pretend):
        prefix = self.__class__.__name__ + '_'
        assert type(name_pretend) == str, 'Name need be str'
        assert bool(re.fullmatch(r'\w+', name_pretend)), 'Name have to consists of alphas, nums and _'
        assert name_pretend.startswith(prefix), 'Name have to begin from className_'
        assert not OI.checkNameRepeating(self.__class__, name_pretend), 'Name {} is already exists'.format(name_pretend)
        self._Name = name_pretend
        
    def strInitialization(self, new_attribs_dict):
        curr_attribs_dict = self.__dict__
        for attr_name, attr_new_value in new_attribs_dict.items():
            if attr_name == 'Name':
                continue
            assert ('_' + attr_name) in curr_attribs_dict.keys(), 'Attrib {} not found'.format(attr_name)
            if type(attr_new_value) == str:
                setattr(self, '_' + attr_name, attr_new_value)
            else:
                setattr(self, attr_name, attr_new_value)
            
    def strEvaluation(self):
        # automatically works only for properties... thats enough
        cls_ierarh = reversed(self.__class__.__mro__[:-1])
        for cls in cls_ierarh:
            cls_prop_names = list(filter(lambda i: type(cls.__dict__[i]) == property, cls.__dict__.keys()))
            for prop_name in cls_prop_names:
                if prop_name == 'Name':
                    continue
                assert '_' + prop_name in self.__dict__.keys() , 'Prop {} not in obj'.format(prop_name)
                setattr(self, prop_name, getattr(self, '_' + prop_name))
    
    @classmethod
    def allSubclasses(cls):
        subclses = cls.__subclasses__()
        for subcls in subclses:
            subclses.extend(subcls.allSubclasses())
        return subclses

In [9]:
# ! Most Base class - inits first !
class BasisLine:
    pass

# Basis = BasisLines + BasisCSs
class Basis(BFP):
    File = 'BASIS'
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self._MainCoordSystem = None
        self._CoordSystems = []
        self._BasisLines = []
        
        self.strInitialization(kwargs)
        
    @property
    def MainCoordSystem(self):
        return self._MainCoordSystem
    @MainCoordSystem.setter
    def MainCoordSystem(self, value):
        self._MainCoordSystem = CE.simpleSingleCast(value, CoordinateSystem)
        
    @property
    def CoordSystems(self):
        return self._CoordSystems
    @CoordSystems.setter
    def CoordSystems(self, value):
        self._CoordSystems = CE.simpleIterableCast(value, list, CoordinateSystem)
        
    @property
    def BasisLines(self):
        return self._BasisLines
    @BasisLines.setter
    def BasisLines(self, value):
        self._BasisLines = CE.simpleIterableCast(value, list, BasisLine)
        
    def addCoordSystem(self, value):
        self._CoordSystems.append(CE.simpleSingleCast(value, CoordinateSystem))
        
    def removeCoordSystem(self, value):
        self._CoordSystems.remove(value)
        
    def addBasisLine(self, value):
        self._BasisLines.append(CE.simpleSingleCast(value, BasisLine))
        
    def removeBasisLine(self, value):
        self._BasisLines.remove(value)

# Number of base CS = number of MainFrames
class CoordinateSystem(BFP):
    File = 'COORD_SYSTEMS'
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self._FieldCoord = None
        self._RelCsCoDirect = None
        self._CoordToBasisTransformation = None #CoordToBasisTransformation(0,1)
        self._Basis = None
        
        self.strInitialization(kwargs)
        
    @property
    def FieldCoord(self):
        return self._FieldCoord
    @FieldCoord.setter
    def FieldCoord(self, value):
        self._FieldCoord = CE.simpleSingleCast(value, FieldCoord)
        
    @property
    def RelCsCoDirect(self):
        return self._RelCsCoDirect
    def RelCsCoDirectChecker(self, value):
        return abs(int(value)) == 1
    @RelCsCoDirect.setter
    def RelCsCoDirect(self, value):
        self._RelCsCoDirect = CE.customCast(value, self.RelCsCoDirectChecker)
        
    @property
    def CoordToBasisTransformation(self):
        return self._CoordToBasisTransformation
    @CoordToBasisTransformation.setter
    def CoordToBasisTransformation(self, value):
        self._CoordToBasisTransformation = CE.simpleSingleCast(value, CoordToBasisTransformation)
        
    @property
    def Basis(self):
        return self._Basis
    @Basis.setter
    def Basis(self, value):
        self._Basis = CE.simpleSingleCast(value, Basis)
           
    def basisEvaluations(self, main_in_basis = False):
        if main_in_basis:
            self.Basis = main_in_basis
            self.CoordToBasisTransformation = CoordToBasisTransformation(0,1)
        else:
            relCsSystemCBT = self.FieldCoord.Cs.CoordToBasisTransformation
            self.Basis = self.FieldCoord.Cs.Basis
            self.CoordToBasisTransformation = CoordToBasisTransformation(relCsSystemCBT.Shift + self.FieldCoord.X,\
                                                                     relCsSystemCBT.Direction * self.RelCsCoDirect)
      
    
class Point(BFP):
    File = 'BASEPOINTS'
    def __init__(self,**kwargs): 
        super().__init__(**kwargs)
        self._OnPoint = None # Point
        self._FieldCoord = None
        self._Line = None # Line
        
        #self._System = None # CoordinateSystem
        #self._PropBorders = None # CustomType
        #self._IsFictive = False # bool
        #self._AbsoluteCoord = None
        #self._cPoint = None
        
        self.strInitialization(kwargs)
        
    @property
    def OnPoint(self):
        return self._OnPoint
    @OnPoint.setter
    def OnPoint(self, value):
        self._OnPoint = CE.simpleSingleCast(value, Point)
        
    @property
    def FieldCoord(self):
        return self._FieldCoord
    @FieldCoord.setter
    def FieldCoord(self, value):
        self._FieldCoord = CE.simpleSingleCast(value, FieldCoord)
       
    '''  
    def evalAC(self):
        baseTT = self.System.TransformationTuple
        self.AbsoluteCoord = baseTT.shift + baseTT.direction * self.LocalCoord
        
    def evalCoordInCS(self, CS, jumper):
        pass
    
    def toContiLayer(self, contiPoint = None):
        if contiPoint is None:
            cpnt = cPoint()
            cpnt.addIPnt(self)
            self.cPoint = cpnt
        else:
            contiPoint.addIPnt(self)
            self.cPoint = contiPoint
    ''' 
    
class Line(BFP):
    File = 'BASELINES'
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self._Point_Begin = None 
        self._Point_End = None
        self._Angle = None # int ?
        self._Order = None # int 
        self._PointsList = None 
        self._cElemsSet = set() 
        
        self.strInitialization(kwargs)
        
    '''
    def toContiLayer(self):
        cel = cElement()
        cel.inputLine = self
        self.cElemsSet.add(cel)
    '''
        
class Light(BFP):
    File = 'LIGHTS'
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self._PlacePoint = None
        self._StartRoutePoints = None  # list
        self._Type = None
        self._Direction = None
        self._Colors = None  # list
        self._MainSignal = None
        
        self.strInitialization(kwargs)
        

In [10]:
class Pipeline:
    
    def resetOI(self):
        OI.removeAllClassInstances()
        
    def initObjsFromFiles(self):
        rootPath = os.getcwd()
        
        classes = BFP.allSubclasses()
        for cls in classes:
            tree = os.walk(rootPath)
            if 'File' in cls.__dict__.keys():
                assert type(cls.File) == str, 'File name need be str'
                for d, dirs, files in tree:
                    for file in files:
                        if len(file)>3 and file[-4:]=='.csv':
                            file_short_name = file[:-4]
                            file_fullPath = d + '\\' + file
                            if file_short_name == cls.File:
                                df = pd.read_csv(file_fullPath, sep=';', dtype='str', keep_default_na=False)
                                for index in df.index:
                                    attribsDict = {}
                                    for column in df.columns:
                                        if column=='Npp':
                                            continue
                                        attribsDict[column] = df.loc[index,column]
                                    cls(**attribsDict)
    
    def evaluateObjAttribs(self):
        classes = BFP.allSubclasses()
        for cls in classes:
            objs = OI.getInstances(cls)
            for obj in objs:
                obj.strEvaluation()
                
    def evaluateBases(self):
        bases = OI.getInstances(Basis)
        main_css = dict({(basis.MainCoordSystem, basis) for basis in bases})
        css = OI.getInstances(CoordinateSystem)
        for cs in css:
            if cs in main_css.keys():
                cs.basisEvaluations(main_css[cs])
            else:
                cs.basisEvaluations()
   
    def writeDataFile(self, cls, file_name, folder):
        rootPath = os.getcwd()
        if not (folder in os.listdir(path = rootPath)):
            os.mkdir(rootPath + '\\' + folder)
        currDir = rootPath + '\\' + folder
            
        columns = ['Npp'] 
        cls_ierarh = reversed(cls.__mro__[:-1])
        for cls1 in cls_ierarh:
            cls_prop_names = list(filter(lambda i: type(cls1.__dict__[i]) == property, cls1.__dict__.keys()))
            for prop_name in cls_prop_names:
                columns.append(prop_name)
                
        Npp = 0
        rows = []
        objs = OI.getInstances(cls)
        for obj in objs:
            for column in columns:
                if column == 'Npp':
                    Npp += 1
                    new_row = [str(Npp)]
                else:
                    attr_val = getattr(obj, column)
                    if not attr_val is None:
                        attr_val = str(attr_val)
                    new_row.append(attr_val)
            rows.append(new_row)
        df = pd.DataFrame(rows, columns=columns) 
        df.to_csv('{}\\{}.csv'.format(currDir, file_name), sep = ';', index=False)
                                    
PPL = Pipeline()                

In [11]:
PPL.resetOI()
PPL.initObjsFromFiles()
PPL.evaluateObjAttribs()
PPL.evaluateBases()

In [19]:
a = dict({(i, 1) for i in [1,2,3,4,4,5,5]})
a

{3: 1, 2: 1, 5: 1, 4: 1, 1: 1}

In [17]:
PPL.writeDataFile(CoordinateSystem, 'coord_systems', 'super_Folder')

In [20]:
PPL.writeDataFile(Point, 'points', 'super_Folder')

In [19]:
PPL.writeDataFile(Light, 'lights', 'super_Folder')

In [12]:
[i for i in OI.getInstances(Point)] # CoordinateSystem Point Line Light

[Point_Link_Obuh_Kirov_1,
 Point_Link_Ribats_Kirov_1,
 Point_Link_Ribats_PeregIzori_2,
 Point_Link_Izori_PeregIzori_2,
 Point_Link_Obuh_Kirov_2,
 Point_Link_Obuh_Kirov_3,
 Point_Link_Obuh_Kirov_4,
 Point_Link_Ribats_Kirov_2,
 Point_Link_Ribats_Kirov_3,
 Point_Link_Ribats_Kirov_4,
 Point_Link_Ribats_PeregIzori_1,
 Point_Link_Ribats_Izori_3,
 Point_Link_Ribats_PeregIzori_4,
 Point_Link_Izori_PeregIzori_1,
 Point_Link_Izori_PeregIzori_4,
 Point_Link_Kupch_PeregKupch,
 Point_Link_Ribats_PeregKupch,
 Point_Link_Ribats_Tpk_19,
 Point_Link_Ribats_Tpk_17,
 Point_Link_Ribats_Pod_15,
 Point_Link_Ribats_Tpk_5,
 Point_Link_Ribats_Tpk_9,
 Point_Link_Ribats_Tpk_10,
 Point_Link_Ribats_Pod_23,
 Point_Link_Ribats_Pod_24,
 Point_Link_Ribats_Pod_33,
 Point_Link_Ribats_Pod_21,
 Point_s10,
 Point_s8,
 Point_s16,
 Point_s28,
 Point_s34,
 Point_s32,
 Point_s38,
 Point_s23,
 Point_s13,
 Point_s9,
 Point_s3,
 Point_s2,
 Point_s6,
 Point_s4,
 Point_s12,
 Point_s14,
 Point_s18,
 Point_s22,
 Point_s20,
 Point_s24

In [12]:
Basis_Station.MainCoordSystem

CoordinateSystem_Basis_1

In [17]:
Point_Link_Ribats_Kirov_1.FieldCoord.toBasis()

FieldCoord((-1043.0,None),CoordinateSystem_Basis_1)

In [15]:
# (Point_Link_Ribats_Kirov_1.FieldCoord + 100).PicketCoord

In [13]:
CoordinateSystem_Picket_AB.__dict__

{'_Name': 'CoordinateSystem_Picket_AB',
 '_FieldCoord': FieldCoord((-14144.0,None),CoordinateSystem_EC),
 '_RelCsCoDirect': 1,
 '_CoordToBasisTransformation': CoordToBasisTransformation(-14144.0,1),
 '_Basis': Basis_Station}

In [15]:
class A:
    def __init__(self):
        self.val = 3
        
    def __mul__(self,other):
        assert type(other) == B
        return self.val * other.val
    
class B:
    def __init__(self):
        self.val = 4

In [17]:
a = A()
b = B()
b*a

TypeError: unsupported operand type(s) for *: 'B' and 'A'

In [98]:
#Point_Link_Ribats_Kirov_3.FieldCoord.Cs

In [26]:
#Point_Link_Ribats_Tpk_19.FieldCoord = FieldCoord((6,2),CoordinateSystem_EC,E_FieldAxis.X)

In [12]:
Point_Link_Ribats_Tpk_19.FieldCoord

FieldCoord((410.0,None),CoordinateSystem_EC,E_FieldAxis.X)

In [15]:
# Graph classes
class Pnt(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._CnctGroup = None
        
        self.strInitialization(kwargs)
        self.strEvaluation()
    
    @property
    def CnctGroup(self):
        return self._CnctGroup
    @CnctGroup.setter
    def CnctGroup(self, value):
        self._CnctGroup = CE.simpleSingleCast(value, CnctGroup)
    
class Elt(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._Pnts = None
        
        self.strInitialization(kwargs)
        self.strEvaluation()
        
    @property
    def Pnts(self):
        return self._Pnts
    @Pnts.setter
    def Pnts(self, value):
        self._Pnts = CE.simpleIterableCast(value, tuple, Pnt, 2)

class Cnct(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._Pnt = None
        self._Elts = None
        self._Active = None
        self._CnctGroup = None
        self._MergePolicy = None
        
        self.strInitialization(kwargs)
        self.strEvaluation()
    
    @property
    def Pnt(self):
        return self._Pnt
    @Pnt.setter
    def Pnt(self, value):
        self._Pnt = CE.simpleSingleCast(value, Pnt)
    
    @property
    def Elts(self):
        return self._Elts
    @Elts.setter
    def Elts(self, value):
        self._Elts = CE.simpleIterableCast(value, tuple, Elt, 2)
    
    @property
    def Active(self):
        return self._Active
    @Active.setter
    def Active(self, value):
        self._Active = CE.simpleSingleCast(value, bool)
    
    @property
    def CnctGroup(self):
        return self._CnctGroup
    @CnctGroup.setter
    def CnctGroup(self, value):
        self._CnctGroup = CE.simpleSingleCast(value, CnctGroup)
    
    @property
    def MergePolicy(self):
        return self._MergePolicy
    @MergePolicy.setter
    def MergePolicy(self, value):
        self._MergePolicy = CE.enumSimpleCast(value, E_MergePolicy)

class CnctGroup(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._Pnt = None
        self._Cncts = None
        
        self.strInitialization(kwargs)
        self.strEvaluation()
    
    @property
    def Pnt(self):
        return self._Pnt
    @Pnt.setter
    def Pnt(self, value):
        self._Pnt = CE.simpleSingleCast(value, Pnt)
        
    @property
    def Cncts(self):
        return self._Cncts
    @Cncts.setter
    def Cncts(self, value):
        self._Cncts = CE.simpleIterableCast(value, set, Cnct)
        
    def addConnection(self, cnct):
        assert issubclass(type(cnct),Cnct), 'cnct isnt Cnct type'
        self._Cncts.add(cnct)
        
    def removeConnection(self, cnct):
        self._Cncts.remove(cnct)
    
# Grph = Connections manager
class Grph(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._Pnts = set()
        self._Elts = set()
        
        self.strInitialization(kwargs)
        self.strEvaluation()
        
    @property
    def Pnts(self):
        return self._Pnts 
    @Pnts.setter
    def Pnts(self, value):
        self._Pnts = CE.simpleIterableCast(value, set, Pnt)
        
    @property
    def Elts(self):
        return self._Elts
    @Elts.setter
    def Elts(self, value):
        self._Elts = CE.simpleIterableCast(value, set, Elt)
        
    def _addPnt(self, pnt):
        self._Pnts.add(pnt)
        
    def _addElt(self, elt):
        self._Elts.add(elt)
            
    def createPnt(self):
        pnt = Pnt()
        self._addPnt(pnt)
        return pnt
            
    def createElt(self, pnts):
        elt = Elt(Pnts=pnts)
        self._addElt(elt)
        return elt
    
    def extendConnGroup(self, pnt, cnct):
        if cnct.Pnt.CnctGroup is None:
            cg = CnctGroup(Pnt = cnct.Pnt, Cncts = {cnct})
            cnct.Pnt.CnctGroup = cg
            cnct.CnctGroup = cg
        else:
            cg = cnct.Pnt.CnctGroup
            cg.addConnection(cnct)
            cnct.CnctGroup = cg
    
    def createConnection(self, elt_1, elt_2, pnt_common, merge_policy = E_MergePolicy.Side, activity=False):
        assert issubclass(type(elt_1), Elt), 'Elt inst is needed, given inst is {}'.format(elt_1)
        assert issubclass(type(elt_2), Elt), 'Elt inst is needed, given inst is {}'.format(elt_2)
        assert issubclass(type(pnt_common), Pnt), 'Pnt inst is needed, given inst is {}'.format(pnt_common)
        assert type(activity) == bool, 'Bool is needed, given inst is {}'.format(activity)
        
        assert (pnt_common in elt_1.Pnts), 'Pnt for merge not in element 1: {}'.format(elt_1.Pnts)
        assert (pnt_common in elt_2.Pnts), 'Pnt for merge not in element 2: {}'.format(elt_2.Pnts)
        
        cnct = Cnct(Pnt = pnt_common, Elts = (elt_1, elt_2), MergePolicy = merge_policy, Active = activity)
        self.extendConnGroup(pnt_common, cnct)
        return cnct
            
    def splitElt(self, elt, pnt = None):
        assert issubclass(type(elt), Elt), 'Elt inst is needed, given inst is {}'.format(elt)
        if pnt is None:
            pnt = Pnt()
        else:
            assert issubclass(type(pnt), Pnt), 'Pnt inst is needed, given inst is {}'.format(pnt)
        new_elt_1 = Elt(Pnts = (elt.Pnts[0], pnt))
        new_elt_2 = Elt(Pnts = (elt.Pnts[1], pnt))
        self.createConnection(new_elt_1, new_elt_2, pnt, E_MergePolicy.Direct)
        OI.removeObj(elt)
        return (new_elt_1, new_elt_2), pnt
        
    def connectEltToElt(self, elt_1, pnt_elt_1, elt_2):
        assert issubclass(type(elt_1), Elt), 'Elt inst is needed, given inst is {}'.format(elt_1)
        assert issubclass(type(elt_2), Elt), 'Elt inst is needed, given inst is {}'.format(elt_2)
        assert issubclass(type(pnt_elt_1), Pnt), 'Pnt inst is needed, given inst is {}'.format(pnt_elt_1)
        new_elts_2, _ = self.splitElt(elt_2, pnt_elt_1)
        new_elts_2_1, new_elts_2_2 = new_elts_2
        self.createConnection(new_elts_2_1, elt_1, pnt_elt_1)
        self.createConnection(new_elts_2_2, elt_1, pnt_elt_1)
        
        

In [16]:
Grph(Name = 'Grph_Probe')

Grph_Probe

In [17]:
Grph_Probe.createPnt()
Grph_Probe.createPnt()
Grph_Probe.createPnt()
Grph_Probe.createPnt()

Pnt_4

In [18]:
Grph_Probe.createElt((Pnt_1,Pnt_2))
Grph_Probe.createElt((Pnt_3,Pnt_2))
Grph_Probe.createElt((Pnt_3,Pnt_4))

Elt_3

In [19]:
Grph_Probe.connectEltToElt(Elt_1, Pnt_1, Elt_3)

In [20]:
[i for i in OI.getInstances(Elt)]

[Elt_1, Elt_2, Elt_4, Elt_5]

In [21]:
Pnt_1.__dict__

{'_Name': 'Pnt_1', '_CnctGroup': CnctGroup_1}

In [22]:
CnctGroup_1.__dict__

{'_Name': 'CnctGroup_1', '_Pnt': Pnt_1, '_Cncts': {Cnct_1, Cnct_2, Cnct_3}}

In [23]:
[i for i in OI.getInstances(Cnct)]

[Cnct_1, Cnct_2, Cnct_3]

In [26]:
Cnct_3.__dict__

{'_Name': 'Cnct_3',
 '_Pnt': Pnt_1,
 '_Elts': (Elt_5, Elt_1),
 '_Active': False,
 '_CnctGroup': CnctGroup_1,
 '_MergePolicy': 'E_MergePolicy.Side'}

In [34]:
Grph_Probe.splitElt(Elt_1)

((Elt_2, Elt_3), Pnt_3)

In [ ]:
# 2D - Geometry operations

In [2]:
'+' in 'dfgdkjhgdkffgd'

False

In [79]:
a

FieldCoord(12923,CoordinateSystem_Basis_1)

In [ ]:
class Point_2D(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._X = None
        self._Y = None
        
        self.strInitialization(kwargs)
        self.strEvaluation()
        
    @property
    def X(self):
        return self._X
    @X.setter
    def X(self, Value):
        self._X = CE.simpleSingleCast(value, float)
    @property
    def Y(self):
        return self._Y
    @X.setter
    def Y(self, Value):
        self._Y = CE.simpleSingleCast(value, float)
        
class Line2D(BFP):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.strInitialization(kwargs)
        self.strEvaluation()

In [ ]:
class Rectangle:
    def __init__(self, lb, rt): # left bottom, right top
        self.lb = (lb[0],lb[1])
        self.lt = (lb[0],rt[1])
        self.rt = (rt[0],rt[1])
        self.rb = (rt[0],lb[1])
    
    def includes(self,pnt):
        return (self.lb[0] <= pnt[0] < self.rt[0]) and (self.lb[1] <= pnt[1] < self.rt[1])
        
    def getCorners(self):
        return [self.lb , self.lt , self.rt , self.rb]

In [ ]:
class StraightLine:
    def __init__(self, pnt1, pnt2):
        if pnt1[0] > pnt2[0]:
            (pnt1, pnt2) = (pnt2, pnt1)
        self.pnt1 = pnt1
        self.pnt2 = pnt2
        self.k = (pnt2[1] - pnt1[1]) / (pnt2[0] - pnt1[0])
        self.b = (pnt1[1] * pnt2[0] - pnt1[0] * pnt2[1]) / (pnt2[0] - pnt1[0])
        
    def getParams(self):
        return self.k, self.b
    
    def evalY(self, X):
        return self.k * X + self.b

#ml = StraightLine((0,1),(1,0))
#print(ml.getParams())
#print(ml.evalY(-1))

In [ ]:
class Point_2D:
    def __init__(self,Name,X,Y):
        self.Name = Name
        self.X = X
        self.Y = Y
        
#myp = Point_2D('myp',40,50)
#myp.X

In [ ]:
# min_X = np.inf
# max_X = - np.inf
min_X = min([pnt.AbsoluteCoord for pnt in Point.getInstances()])
max_X = max([pnt.AbsoluteCoord for pnt in Point.getInstances()])
min_Y = min([gl.Shift for gl in GroundLine.getInstances() if not gl.Shift is None])
max_Y = max([gl.Shift for gl in GroundLine.getInstances() if not gl.Shift is None])     
print(min_X,max_X) 
print(min_Y,max_Y)

In [ ]:
pointsDrawList = [] # [(,,) , (,,) , (,,)]
for pnt in Point.getInstances():
    line = pnt.Line
    if not (line in GroundLine.getInstances()): 
        continue
    if not (line.Shift is None):
        pointsDrawList.append((pnt.Name,pnt.AbsoluteCoord,max_Y + min_Y - line.Shift))
    #else:
        

In [ ]:
boxstyle = BoxStyle("Square", pad=0)
props = {'boxstyle': boxstyle,
         'facecolor': 'none', # 'white'
         'linestyle': 'solid',
         'linewidth': 1,
         'edgecolor': 'none'}  # 'black'

In [ ]:
font = FontProperties()
font.set_family('serif')
font.set_name('Times New Roman')
font.set_size('6')
#font.set_style('italic')
#font.set_stretch('500')

width = 1200 # in points
height = 800 # in points

pix_to_y = (max_Y - min_Y) / height

dpi = 100
textBeginDistance = 0.5

GlobalFigure = plt.figure(figsize=(width//dpi, height//dpi), dpi=dpi) #,edgecolor = 'black'
GlobalAxis = GlobalFigure.add_subplot(111)
#GlobalAxis.xaxis.set(ticks=np.arange(-10,2+0.25,0.25))
#GlobalAxis.yaxis.set(ticks=np.arange(-10,2+0.25,0.25))
#(min_X,max_X) = (-2000,2000)
GlobalAxis.set_xlim(min_X, max_X)
GlobalAxis.set_ylim(min_Y, max_Y)

rect_List = []
delta_Dict = {}

subList = pointsDrawList #[:27]
subList = [(a[0].replace('Point','p'),a[1],a[2]) for a in subList]

# Rough drawing
intersect_Correction = False
for drawPoint in subList:
    delta_Dict[drawPoint] = textBeginDistance
    if intersect_Correction:
        need_to_find_intersect = True
        while need_to_find_intersect:
            #print('Current text is ',drawPoint[0])
            currY = drawPoint[2] + delta_Dict[drawPoint]
            need_to_find_intersect = False
            txt = GlobalAxis.text(drawPoint[1],currY,\
                                  drawPoint[0],horizontalalignment='center',\
                                  fontproperties=font, bbox=props)
            plt.draw()
            curr_rect = Rectangle(txt.get_bbox_patch().get_extents().p0,txt.get_bbox_patch().get_extents().p1)
            for rect in rect_List:
                for corner in curr_rect.getCorners():
                    if rect.includes(corner):
                        #print('Include is found')
                        need_to_find_intersect = True
                        #print('Up on ',txt.get_bbox_patch().get_extents().height)
                        delta_Dict[drawPoint] += (txt.get_bbox_patch().get_extents().height * pix_to_y)
                        break
                if need_to_find_intersect:
                    break
        rect_List.append(curr_rect)
                    
GlobalAxis.clear()
GlobalAxis.set_xlim(min_X, max_X)
GlobalAxis.set_ylim(min_Y, max_Y)
    
# Сlean drawing 

for drawPoint in subList: 
    GlobalAxis.plot(drawPoint[1],drawPoint[2], color='black', marker='o', markersize=3)   
    txt = GlobalAxis.text(drawPoint[1],drawPoint[2] + delta_Dict[drawPoint],\
                          drawPoint[0],horizontalalignment='center',\
                          fontproperties=font, bbox=props)           


        
    #plt.draw()
    #print(drawPoint[0], txt.get_bbox_patch().get_extents().width, txt.get_bbox_patch().get_extents().height)
    
# ,style = 'italic', fontweight ="bold"
plt.grid()
#plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
#plt.savefig('foo.png')
plt.show()

In [ ]:
r = Rectangle(txt.get_bbox_patch().get_extents().p0,txt.get_bbox_patch().get_extents().p1)

In [ ]:
r.getCorners()

In [ ]:
txt.get_bbox_patch().get_extents().p0

In [ ]:
txt.get_bbox_patch().get_extents().p1

In [ ]:
r.includes([500,400])

In [ ]:
help(Bbox)

In [ ]:
width = 12
height = 12
dpi = 100
pnt1 = (0.2,0.2)
pnt2 = (0.8,0.8)
pnt3 = (1.5,0.2)
pnt4 = (0.75,0.5)
GlobalFigure = plt.figure(figsize=(width, height), dpi=dpi)
GlobalAxis = GlobalFigure.add_subplot(111)
GlobalAxis.set_xlim(-1, 2)
GlobalAxis.set_ylim(-1, 2)
l = mlines.Line2D([pnt1[0],pnt2[0]],[pnt1[1],pnt2[1]], color='black', marker='o', markersize=6, linewidth=3)
l1 = mlines.Line2D([pnt1[0],pnt3[0]],[pnt1[1],pnt3[1]], color='blue', marker='o', markersize=6, linewidth=3)
l2 = mlines.Line2D([pnt2[0],pnt3[0]],[pnt2[1],pnt3[1]], color='red', marker='o', markersize=6, linewidth=3)
GlobalAxis.add_line(l)
GlobalAxis.add_line(l1)
GlobalAxis.add_line(l2)
GlobalAxis.plot(pnt4[0],pnt4[1], color='black', marker='o', markersize=6)
#plt.savefig('foo.png')
GlobalAxis.xaxis.set(ticks=np.arange(-1,2+0.25,0.25))
GlobalAxis.yaxis.set(ticks=np.arange(-1,2+0.25,0.25))

textDistance = 0.02
GlobalAxis.text(pnt1[0],pnt1[1] + textDistance,'Pnt1',horizontalalignment='center',fontname = 'Courier') #  + textDistance
GlobalAxis.text(pnt2[0],pnt2[1] + textDistance,'Pnt2',horizontalalignment='center')
GlobalAxis.text(pnt3[0],pnt3[1] + textDistance,'Pnt3',horizontalalignment='center')
GlobalAxis.text(pnt4[0],pnt4[1] + textDistance,'Pnt4',horizontalalignment='center')

plt.grid()
plt.show()

In [ ]:
# CUT 1D optimization for convex functions
def _1D_cutOptimization(func, borders, maxormin, precision, addit_args = []):
    assert maxormin in ['min', 'max']
    curr_borders = (min(borders), max(borders))
    curr_region_size = curr_borders[1] - curr_borders[0]
    if maxormin == 'min':
        k = 1
    else:
        k = -1
    while curr_region_size > precision:
        border_vals = k * func(curr_borders[0], *addit_args), k *func(curr_borders[1], *addit_args)
        if border_vals[1] > border_vals[0]:
            curr_borders = curr_borders[0], 0.5*(curr_borders[0] + curr_borders[1])
        else:
            curr_borders = 0.5*(curr_borders[0] + curr_borders[1]), curr_borders[1]
        curr_region_size/=2 
        
        curr_f_value = k * min(border_vals)
        curr_x_value = curr_borders[0]
        
    return curr_x_value, curr_f_value

#def my_func(x):
#    return (x-4)**2 + 2
#_1D_cutOptimization(my_func, [0,3], 'min', 0.01)

In [ ]:
#math.degrees(math.pi/2)
#math.tan(math.radians(89))
#math.degrees(math.atan(1/3))

In [ ]:
A = np.array([[4, 3, 2], [-2, 2, 3], [3, -5, 2]]) #
B = np.array([25, -10, -4])
np.linalg.solve(A,B)

In [ ]:
# GEOMETRY EVALUATIONS
def floatFrac(a, n):
    assert type(n)==int
    k = 1 if (a >= 0) else -1
    frac_part = math.modf(a)[0]
    int_part = int(math.modf(a)[1])
    return frac_part + k * (k * int_part%n)

# angle to (-90, 90]
def angleToRange_m90_p90(ang):
    ang = floatFrac(ang,180)
    if ang <= -90:
        ang+=180
    elif ang > 90:
        ang-=180
    return ang

In [ ]:
# For given point (x,y) and Angle in point (in degrees) 
# function returns params A1,B1 in equation A1*x + B1*y = C1 
# with conditions A1^2 + B1^2 = 1 and B1 >= 0 
def directLineParams(x, y, ang):
    ang = angleToRange_m90_p90(ang)
    ang = math.radians(ang)
    if abs(math.tan(ang)) > 1e+6:
        #print('Special Case !1')
        #print(math.tan(math.radians(ang)))
        (A1, B1, C1) = (1, 0, x)
    else:
        B1 = abs(math.cos(ang))
        A1 = - B1 * math.tan(ang)
        C1 =  B1 * (y - math.tan(ang) * x)
    return (A1, B1, C1)

In [ ]:
def linesIntersection(params1,params2):
    if (params1[0] - params2[0])**2 + (params1[1] - params2[1])**2 < 1e-13:
        raise ValueError('Lines is Approxi-Parallel ! No intersection')
    else:
        A = np.array([[params1[0],params1[1]], [params2[0],params2[1]]])
        B = np.array([params1[2],params2[2]])
        return tuple(np.linalg.solve(A,B))

#line_1 = directLineParams(1, 1, 1)
#line_2 = directLineParams(3, 4, 2)
#linesIntersection(line_1,line_2)

In [ ]:
def curvatureOfBezier(t, pnt1, pnt2, pnt3):
    x1, y1 = pnt1
    x2, y2 = pnt2
    x3, y3 = pnt3
    return abs(0.5*(-(2*(x3-x1) + x1 - x2)*(2*(y3-y1)*t - (y3-y1) + t*y1 - t*y2) + \
                (2*(y3-y1) + y1 - y2)*(2*(x3-x1)*t - (x3-x1) + t*x1 - t*x2)) * \
                ((2*(x3-x1)*t - (x3-x1) + t*x1 - t*x2)**2 + (2*(y3-y1)*t - (y3-y1) + t*y1 - t*y2)**2)**(-1.5))

def supportF_CurvatureOfBezier(t, pnt1, pnt2, pnt3):
    return -curvatureOfBezier(t, pnt1, pnt2, pnt3)

#Pnt1,Pnt2,Pnt3 = (0,0),(1,1),(1,0)
#curvatureOfBezier(0.8, Pnt1, Pnt2, Pnt3)

In [ ]:
def distance_2points(pnt1, pnt2):
    return ((pnt2[0] - pnt1[0])**2 + (pnt2[1] - pnt1[1])**2)**0.5
#distance2points((0,0),(3,4))

In [ ]:
# function returns angle of line, connecting two points
# and (+1) or (-1) direction from pnt 1 to pnt2 (+ is x2 > x1 besides 90 deg, y2 > y1)
def angleAndDirection_2Points(pnt1, pnt2):
    ang = 90
    direc = int((int(pnt2[1] > pnt1[1]) - 0.5) * 2)
    if abs(pnt2[0] - pnt1[0]) > 1e-3:
        ang = math.degrees(math.atan((pnt2[1] - pnt1[1]) / (pnt2[0] - pnt1[0])))
        direc = int((int(pnt2[0] > pnt1[0]) - 0.5) * 2)
    #else:
    #    print('Special Case !2')
    return ang, direc
#angleAndDirectionTwoPoints((0,0),(1,0))

In [ ]:
def _2ndPointOnLine(delta, pnt1, ang1):
    #ang1 = floatFrac(ang1,180)
    ang1 = angleCast(ang1)
    if abs(ang1 - 90) < 1e-3:
        #print('Special Case !3')
        pnt0 = pnt1[0], pnt1[1] + delta
    else:
        pnt0 = pnt1[0] + delta * math.cos(math.radians(ang1)), pnt1[1] + delta * math.sin(math.radians(ang1))
    return pnt0
#_2ndPointOnLine(2,(1,1),45)

In [ ]:
def max_1Line_BezierCurvature(pnt1, ang1, pnt2, ang2):
    ang1 = angleCast(ang1)
    ang2 = angleCast(ang2)
    line_1 = directLineParams(pnt1[0], pnt1[1], ang1)
    line_2 = directLineParams(pnt2[0], pnt2[1], ang2)
    pnt3 = linesIntersection(line_1, line_2)
    #print(pnt1,pnt2,pnt3)
    res = _1D_cutOptimization(supportF_CurvatureOfBezier, [0,1], 'min', 0.01, [pnt1, pnt2, pnt3])
    #res = minimize(supportF_CurvatureOfBezier, x0=0.5, bounds=[(0,1)], args=(pnt1, pnt2, pnt3), \
    #               method='L-BFGS-B', options={'gtol':1e-6, 'maxls':5}) #, options={'gtol':1e-09}nelder-mead, options={'xtol': 1e-8, 'disp': True}
    #return res.x[0], res.fun[0]
    return res
#max_1Line_BezierCurvature((1, 1), -45, (3, 4), 45)

In [ ]:
# delta is measuring on (+1) direction of ang1-line
# pnt0 is a point on line ang1 on distance = delta 
def deltaToAngleConversation(delta, pnt1, ang1, pnt2):
    ang1 = angleCast(ang1)
    pnt0 = _2ndPointOnLine(delta, pnt1, ang1)
    ang2 = angleAndDirection_2Points(pnt0, pnt2)[0]
    return max_1Line_BezierCurvature(pnt1, ang1, pnt2, ang2)

def supportF_max1Line_BezierCurvature(delta, pnt1, ang1, pnt2):
    ang1 = angleCast(ang1)
    return -deltaToAngleConversation(delta, pnt1, ang1, pnt2)[1]

In [ ]:
deltaToAngleConversation(-0.315995, (1, 1), -45, (3, 4))

In [ ]:
supportF_max1Line_BezierCurvature(-2.378847, (1, 1), -45, (3, 4))

In [ ]:
# function finds angle in second point which leads to min(max(curvature))
# direction means in which side to plot Bezier curve
def optimalBezierDelta(pnt1, ang1, direct, pnt2):
    ang1 = angleCast(ang1)
    max_delta = distance_2points(pnt1, pnt2)
    if direct > 0:
        x0 = max_delta
        bounds=[1e-5*max_delta, max_delta]
    else:
        x0 = -max_delta
        bounds=[-max_delta, -1e-5 * max_delta]
    #res = minimize(supportF_max1Line_BezierCurvature, x0=x0, \
    #               bounds=bounds, args=(pnt1, ang1, pnt2), \
    #               method='L-BFGS-B', options={'gtol':1e-6, 'maxls':5}) #, options={'gtol':0.4e-05}, options={'gtol':1e-09}nelder-mead, options={'xtol': 1e-8, 'disp': True}
    res = _1D_cutOptimization(supportF_max1Line_BezierCurvature, bounds, 'min', 0.001*max_delta, [pnt1, ang1, pnt2])
    
    return res
#optimalBezierDelta((0,0), 30, 1, (100, 1))

In [ ]:
np.argmin([5,2,3,1,5])

In [ ]:
direction =  1
Pnt1, ang1= (0, 0), 0
#Pnt2, ang2= (1, 1), 45
Pnt2, ang2= (33.36, 1.52), 45
line_1 = directLineParams(Pnt1[0], Pnt1[1], ang1)
line_2 = directLineParams(Pnt2[0], Pnt2[1], ang2)

evalRes = optimalBezierDelta(Pnt1, ang1, direction, Pnt2)[0]

Pnt3 = _2ndPointOnLine(evalRes,Pnt1, ang1)
#Pnt3 = linesIntersection(line_1,line_2)
print(Pnt3)
print(angleAndDirection_2Points(Pnt3,Pnt2))
      
x_list = [i[0] for i in [Pnt1,Pnt2,Pnt3]]
y_list = [i[1] for i in [Pnt1,Pnt2,Pnt3]]
#print(x_ses)
#print(y_ses)

# visualization
Path = mpath.Path

max_dimension = 10

assert max(x_list) - min(x_list) != 0
assert max(y_list) - min(y_list) != 0

sizesXY = [max(x_list) - min(x_list), max(y_list) - min(y_list)]
dim_X = max_dimension
dim_Y = max_dimension * (max(y_list) - min(y_list)) / (max(x_list) - min(x_list))
if np.argmax(sizesXY):
    dim_X *= (max(x_list) - min(x_list)) / (max(y_list) - min(y_list))
    dim_Y *= (max(x_list) - min(x_list)) / (max(y_list) - min(y_list))

fig, ax = plt.subplots(figsize=(dim_X, dim_Y), dpi=100)
#plt.figure(figsize=(12, 12), dpi=100)
pp1 = mpatches.PathPatch(
    Path([Pnt1, Pnt3, Pnt2, Pnt1],
         [Path.MOVETO, Path.CURVE3, Path.CURVE3, Path.CLOSEPOLY]),
    fc="none", transform=ax.transData)
#print(list(pp1))

l = mlines.Line2D([Pnt1[0],Pnt2[0]],[Pnt1[1],Pnt2[1]], color='black', marker='o', markersize=6, linewidth=3)
l1 = mlines.Line2D([Pnt1[0],Pnt3[0]],[Pnt1[1],Pnt3[1]], color='blue', marker='o', markersize=6, linewidth=3)
l2 = mlines.Line2D([Pnt2[0],Pnt3[0]],[Pnt2[1],Pnt3[1]], color='red', marker='o', markersize=6, linewidth=3)
ax.add_line(l)
ax.add_line(l1)
ax.add_line(l2)

ax.add_patch(pp1)
ax.set_xlim(min(x_list), max(x_list))
ax.set_ylim(min(y_list), max(y_list))
#ax.set_xlim(-1, max(x_ses))
#ax.set_ylim(-1, max(y_ses))
#ax.plot([0.75], [0.25], "ro")
#ax.set_title('The red point should be on the path')

plt.show()

In [ ]:
#flow_1 = [(1300,6.36), (900,4.9), (700,5.07)] #, (350,12.31)
#flow_2 = [(1300,5.65), (900,5.96), (700,5.5)]
ords = [1300, 900, 700]
flow_1 = [6.36, 4.9, 5.07]
flow_2 = [5.65, 5.96, 5.5]
flow_3 = [11.6, 13.9, 13.06]
flow_4 = [17.3, 17.3, 7.75]

In [ ]:
pnts_X = np.linspace(1, 10.0, 10)

In [ ]:
plt.plot([1, 2, 3, 4, 5], [1, 2, 3, 4, 4])
plt.show()

In [ ]:
math.sin(math.radians(5)) * 61

In [ ]:
#assert 0>1,'message !'